<a href="https://colab.research.google.com/github/divassya/BigDataAnalysis/blob/main/AssiyaKaratay_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Info 
Assignment 4
MET CS777 Big Data Analytics

Faculty - Farshid Alizadeh-Shabdiz, PhD, MBA

Student - Assiya Karatay U95161396 karatay@bu.edu 857-294-7028

#### import libraries

In [ ]:
# Spark installation on Colab
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
# !tar xf spark-3.0.1-bin-hadoop3.2.tgz
# !pip install -q findspark
# !rm -rf spark-3.0.1-bin-hadoop3.2.tgz

!pip install --ignore-installed -q pyspark==3.1.2 

     |████████████████████████████████| 212.4 MB 67 kB/s 
     |████████████████████████████████| 198 kB 55.0 MB/s 


In [15]:
# Set JAVA_HOME and SPARK_HOME
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "spark-3.0.1-bin-hadoop3.2"

# import findspark
# findspark.init("spark-3.0.1-bin-hadoop3.2")# SPARK_HOME


import sys
import requests
from operator import add
import pandas as pd


from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import re
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

#### set up the Google Drive

In [16]:
#### set up the Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
# choose where project files will be saved
project_folder = "/content/drive/MyDrive/CS777_BigDataAnalytics/Assignment4/"
# project_folder = sys.argv[2]
# change the OS to use the project folder as the working directory
os.chdir(project_folder)

print('\n Working directory was changed to ' + project_folder )


 Working directory was changed to /content/drive/MyDrive/CS777_BigDataAnalytics/Assignment4/


### Task 1 Generate a 20K dictionary (10 points)

#### 1.1 The top 20,000 English words
Using Wikipedia pages, find the top 20,000 English words, save them in an array, and sort them based on the frequency of the occurrence.

In [78]:
def parsing(lines):
  # divide the data defore and after url
  division = lines.split('" url')
  # strip all characters except DOC ID
  id = division[0].split('<doc id="')[1] 
  # strip the end of title and last 6 characters containing '.</doc' .
  text = division[1].split('">')[1][:-6]
  # check the UNICODE decoding for readability 
  # and lowercase to count words with the same letters together
  regex = re.compile(r'[^a-zA-Z]', re.UNICODE).split(text.lower())
  return (id,regex)

In [80]:
# 1. get the file
wikiPagesFile= project_folder + "WikipediaPagesOneDocPerLine1000LinesSmall.txt.bz2"
# 2. create an rdd of data
wikiPages = sc.textFile(wikiPagesFile)
# 3. check if each line has id and url
validLines = wikiPages.filter(lambda x : 'id' in x and 'url=' in x)
# 4. trim unnecessary symbols and get (docID, listOfWords) pairs
listOfWords = validLines.map(parsing)
# 5. make every word a tuple dropping DocID
wordsAsTuples = listOfWords.flatMap(lambda x: x[1]).map(lambda x: (x, 1))
# 6. count number of words in a corpus
counts=wordsAsTuples.reduceByKey(lambda x, y: x+y)
# 7. get the top 20K in a corpus
n = 20000
topWords = counts.top(n, lambda x: x[1])
# 8. create an empty RDD that has the number 0 through 20000
emptyRDD = sc.parallelize(range(n))
# 9. order words in descending popularity order, where  
# the word in position 0 is the most common used word
dictionary = emptyRDD.map (lambda x : (topWords[x][0], x))
# 10. save the output 
dictionary.saveAsTextFile(project_folder+'task11_topEnglishWords')
print("Word Positions", dictionary.top(20, lambda x : x[1]))

[('at', 4761), ('two', 1824), ('priesthood', 78), ('in', 27388), ('', 272655)]
Word Positions [('mayhem', 19999), ('hystp', 19998), ('composites', 19997), ('vthl', 19996), ('choisy', 19995), ('ventral', 19994), ('ocelli', 19993), ('humble', 19992), ('lore', 19991), ('fortnights', 19990), ('sunan', 19989), ('mamluk', 19988), ('cartelization', 19987), ('nira', 19986), ('liquidationists', 19985), ('ambivalent', 19984), ('rothbard', 19983), ('tightening', 19982), ('debtor', 19981), ('keynesians', 19980)]


#### 1.2 
As a result, a dictionary has been generated that contains the top 20K most frequent words in the corpus. Next go over each Wikipedia document and check if the words appear in the Top 20K words. At the end, produce an RDD that includes
the docID as key and a Numpy array for the position of each word in the top 20K
dictionary.
(docID, [dictionaryPos1, dictionaryPos2, dictionaryPos3...])

In [70]:
numberOfDocs = wikiPages.count()
numberOfDocs

1000

In [74]:
topWords

[('the', 73600),
 ('of', 34092),
 ('and', 28148),
 ('in', 27383),
 ('to', 22234),
 ('a', 20767),
 ('was', 11986),
 ('as', 8704),
 ('for', 8593),
 ('is', 8315),
 ('on', 8277),
 ('by', 7394),
 ('s', 7039),
 ('with', 6999),
 ('that', 6802),
 ('he', 5420),
 ('from', 5149),
 ('his', 4890),
 ('at', 4761),
 ('it', 4387)]

In [62]:
a = validLines.first()
id = parsing(a)

Word Postions in our Feature Matrix. Last 20 words in 20k positions:  [('it', 19), ('at', 18), ('his', 17), ('from', 16), ('he', 15), ('that', 14), ('with', 13), ('s', 12), ('by', 11), ('on', 10), ('is', 9), ('for', 8), ('as', 7), ('was', 6), ('a', 5), ('to', 4), ('in', 3), ('and', 2), ('of', 1), ('the', 0)]


In [11]:
# sc = SparkContext()

# Set the file paths on your local machine
# Change this line later on your python script when you want to run this on the CLOUD (GC or AWS)

wikiPagesFile= "./WikipediaPagesOneDocPerLine1000LinesSmall.txt.bz2"
# wikiPagesFile = sys.argv[1]
wikiCategoryFile="./wiki-categorylinks-small.csv.bz2"
# wikiCategoryFile = sys.argv[2]

# Read two files into RDDs

wikiCategoryLinks=sc.textFile(wikiCategoryFile)

wikiCats=wikiCategoryLinks.map(lambda x: x.split(",")).map(lambda x: (x[0].replace('"', ''), x[1].replace('"', '') ))

# Now the wikipages
wikiPages = sc.textFile(wikiPagesFile)

# Assumption: Each document is stored in one line of the text file
# We need this count later ... 
numberOfDocs = wikiPages.count()

print(numberOfDocs)
# Each entry in validLines will be a line from the text file
validLines = wikiPages.filter(lambda x : 'id' in x and 'url=' in x)

# Now, we transform it into a set of (docID, text) pairs
keyAndText = validLines.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:-6])) 

#keyAndText.take(1)
#validLines.take(1)

# Now, we split the text in each (docID, text) pair into a list of words
# After this step, we have a data set with
# (docID, ["word1", "word2", "word3", ...])
# We use a regular expression here to make
# sure that the program does not break down on some of the documents

regex = re.compile('[^a-zA-Z]')

# remove all non letter characters
keyAndListOfWords = keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))
# better solution here is to use NLTK tokenizer

# Now get the top 20,000 words... first change (docID, ["word1", "word2", "word3", ...])
# to ("word1", 1) ("word2", 1)...
allWords = keyAndListOfWords.flatMap(lambda x: x[1]).map(lambda x: (x, 1))
allWords.take(10)

# Now, count all of the words, giving us ("word1", 1433), ("word2", 3423423), etc.
allCounts = allWords.reduceByKey(add)
#allCounts.take(10)

# Get the top 20,000 words in a local array in a sorted format based on frequency
# If you want to run it on your laptio, it may a longer time for top 20k words. 
topWords = allCounts.top(20000, lambda x: x[1])

print("Top Words in Corpus:", allCounts.top(10, key=lambda x: x[1]))

# We'll create a RDD that has a set of (word, dictNum) pairs
# start by creating an RDD that has the number 0 through 20000
# 20000 is the number of words that will be in our dictionary
topWordsK = sc.parallelize(range(20000))

# Now, we transform (0), (1), (2), ... to ("MostCommonWord", 1)
# ("NextMostCommon", 2), ...
# the number will be the spot in the dictionary used to tell us
# where the word is located
dictionary = topWordsK.map (lambda x : (topWords[x][0], x))


print("Word Postions in our Feature Matrix. Last 20 words in 20k positions: ", dictionary.top(20, lambda x : x[1]))

Py4JJavaError: ignored

Task 2 - Create the TF-IDF Array (20 Points)
Boston University Metropolitan College
After having the top 20K words we want to create a large array that its
columns are the words of the dictionary with number of occurrences of each word
and the rows are documents.
The first step is calculating the “Term Frequency”, TF (x, w), vector for each
document as follows:
“Term Frequency” is an indication of the number of times a term occurs in a document.
Numerator is number of occurrences of a word, and the denominator is the sum of
all the words of the document.
Next, calculate “Inverse Document Frequency” for all the documents and finally
calculate TF-IDF(w) and create TF-IDF matrix of the corpus:
Note that the “size of corpus” is total number of documents (numerator).
To learn more about TF-IDF see the Wikipedia page:
https://en.wikipedia.org/wiki/Tf-idf
Task 3 - Implement the getPrediction function (30 Points)
Finally, implement the function getPrediction(textInput, k), which will predict the
membership of the textInput to the top 20 closest documents, and the list of top
categories.
You should use the cosine similarity to calculate the distances.
Boston University Metropolitan College
Task 4 – Implement the code using Dataframes (30 points)
Implement the complete code in Dataframe and print out the results of the task 3
using dataframes in pyspark. From the beginning of your code to the end of your
kNN implementation you are allowed to use spark dataframe and python (including
python libraries like numpy). You are not allowed to use RDDs.
Task 5 - Removing Stop Words and Do Stemming (10 points)
Task 5.1 - Remove Stop Words (5 point)
Describe if removing the English Stop words (most common words like ”a,
the, is, are, i, you, ...”) would change the final kNN results.
Does your result change significantly after removing the stop words? Why?
Provide reasons.
You do not need to implement this task.
Task 5.2 – Considering English word stemming (5 point)
We can stem the words [”game”,”gaming”,”gamed”,”games”] to their root
word ”game”.
Does stemming change your result significantly? Why? Provide reasons.
You can learn more about stemming at:
https://en.wikipedia.org/wiki/Stemming
You do not need to implement this task.